# Set-Up

In [1]:
!pip install langchain -q
!pip install langchain_chroma -q
!pip install langchain_community -q
!pip install langchain_groq -q
!pip install grandalf -q
!pip install numpy -q
!pip install pandas -q
!pip install sentence-transformers -q
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
# !pip install ragas

In [22]:
!pip install groqeval

In [3]:
import os

os.environ['GROQ_API_KEY'] = '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'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # annoying transformers warning

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Usage example

## Groq Llama API

In [5]:
youtube_data_path = '/content/drive/MyDrive/itmo-things/psycho_rag_hggg'

In [6]:
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
rag_llm = ChatGroq(model="llama3-8b-8192")  # Used for RAG

<ipython-input-6-180b249cbc5d>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Vectorization

In [7]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
loader = DirectoryLoader(youtube_data_path, use_multithreading=True, loader_cls=TextLoader)
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', ' ', ''],
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

documents = loader.load_and_split(text_splitter=text_splitter)
vectorstore = Chroma.from_documents(documents, embedding=embed_model, collection_name="groq_rag")
retriever = vectorstore.as_retriever()
print(f"Documents indexed: {len(documents)}")

Documents indexed: 517


In [10]:
await retriever.ainvoke("How to fix my ADHD?")

[Document(metadata={'source': '/content/drive/MyDrive/itmo-things/psycho_rag_hggg/Why ADHD is Linked with Addiction [HNje-HuIYdI].txt'}, page_content="things to do and none of them are easy and none of them are simple. At the top of the list is get sober. I'm sorry but you can't convince yourself anymore that drugs are like an acceptable thing to do if you've got ADHD. If you have ADHD that is not well controlled and it's negatively impacting your life, honestly the first thing to do is to get sober. Second thing to do is ideally work with a dual diagnosis clinician who can do therapy to help you work through shame as well as like do CBT around ADHD and stuff like that and can help you like get into the process of recovery. Third thing that you can do is a dopamine detox. So this is going to be really tough for people with ADHD because dopamine detox, the primary problem is essentially boredom. We have a video about dopamine detox that y'all can check out. But dopamine detoxes are goin

## Rag

In [ ]:
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict

In [ ]:
RAG_SYSTEM_PROMPT = """\
You are a psychological assistant for answering questions about mental health. \
You have knowledge base from which you're retrieving some context based on question. \
Use the following pieces of retrieved context given within delimiters to answer the human's questions.
```
{context}
```
Be very careful if you don't know the answer, it's dangerous to give bad answers, just say that you don't know.\
"""

RAG_HUMAN_PROMPT = "{input}"

RAG_PROMPT = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_HUMAN_PROMPT)
])

def format_docs(docs: List[Document]):
    """Format the retrieved documents"""
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs, # Use retriever to retrieve docs from vectorstore -> format the documents into a string
        "input": RunnablePassthrough() # Propogate the 'input' variable to the next step
    }
    | RAG_PROMPT # format prompt with 'context' and 'input' variables
    | rag_llm # get response from LLM using the formatteed prompt
    | StrOutputParser() # Parse through LLM response to get only the string response

)

In [ ]:
await rag_chain.ainvoke("How to be more social if I'm afraid of people?")

'I completely understand your concern. It\'s great that you\'re acknowledging your fear and wanting to work on being more social. Here are some suggestions that might help you:\n\n1. **Start small**: Don\'t try to tackle everything at once. Begin with small, manageable steps. For example, if you\'re afraid of saying hello to someone in class, start by smiling or nodding at them instead. Gradually work your way up to more significant interactions.\n2. **Prepare for internal challenges**: Before trying a new social situation, think about your internal challenges (i.e., what might go wrong in your mind). Anticipate these thoughts and develop a plan to overcome them. For instance, if you\'re worried about saying something stupid, prepare a few conversation topics in advance.\n3. **Focus on others, not yourself**: When interacting with others, try to focus on them rather than your own anxiety. Ask questions, listen actively, and show genuine interest in their lives. This can help take the s

# Class Implementation

In [11]:
from typing import List
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

class PsychoRag:
    def __init__(self, data_path: str):
        # Инициализация модели эмбеддингов и LLM
        self.embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
        self.rag_llm = ChatGroq(model="llama3-8b-8192")

        # history and user question tokens limit
        self.history_max_tokens = 2000
        self.question_max_tokens = 1000


        # Загрузка и разбиение документов
        self.loader = DirectoryLoader(data_path, use_multithreading=True, loader_cls=TextLoader)
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=['\n\n', '\n', ' ', ''],
            chunk_size=2000,
            chunk_overlap=200,
            length_function=len,
            is_separator_regex=False,
        )

        self.documents = self.loader.load_and_split(text_splitter=self.text_splitter)

        self.vectorstore = Chroma.from_documents(self.documents, embedding=self.embed_model, collection_name="groq_rag")
        self.retriever = self.vectorstore.as_retriever()

        self.conversation_history = []

        # Определение промптов
        RAG_SYSTEM_PROMPT = """\
            You are a psychological assistant for answering questions about mental health. \
            You have knowledge base from which you're retrieving some context based on question. \

            Based on conversation history
            '''
            {history}
            '''

            Use the following pieces of retrieved context given within delimiters to answer the human's questions:
            '''
            {context}
            '''

            Be very careful if you don't know the answer, it's dangerous to give bad answers, just say that you don't know.\
            """

        RAG_HUMAN_PROMPT = """\
          {input}
          """

        self.RAG_PROMPT = ChatPromptTemplate.from_messages([("system", RAG_SYSTEM_PROMPT),
         ("human", RAG_HUMAN_PROMPT)])


        self.rag_chain = (
            {
                "context": RunnableLambda(func=lambda x: x['input']) | self.retriever | self.format_docs,
                "history": RunnablePassthrough(),
                "input": RunnablePassthrough()
            }
            | self.RAG_PROMPT
            | self.rag_llm
            | StrOutputParser()
        )

    def format_docs(self, docs: List[Document]):
        return "\n".join(doc.page_content for doc in docs)

    # simple truncating for now
    def preprocess_user_input(self, user_input):
        if len(user_input) > self.question_max_tokens:
            return user_input[: self.question_max_tokens]
        return user_input

    async def ask(self, user_input):
        user_input = self.preprocess_user_input(user_input)
        formatted_history = self._format_history()
        answer = await self.rag_chain.ainvoke({
            "history": formatted_history,
            "input": user_input
        })

        # сохранение истории
        self.conversation_history.append(("User", user_input))
        self.conversation_history.append(("Assistant", answer))

        return answer

    # context window impl
    def _format_history(self):
        lines = []
        total_len = 0
        for speaker, text in reversed(self.conversation_history):
            text_len = len(text.split())
            if total_len + text_len <= self.history_max_tokens:
              lines.append(f"{speaker}: {text}")
              total_len += text_len
            else:
              break

        lines.reverse()
        return "\n".join(lines)

    async def end_session(self):
        """Метод для завершения диалога и формирования отчета"""

        SUMMARY_PROMPT = """\
          You are a psychologist who just completed a session with a client (the user).
          Below is the entire conversation you had with the user.

          Conversation:
          '''
          {history}
          '''

          Based on the entire conversation:
          1. Summarize the user's main psychological concerns and issues that came up.
          2. Provide a few possible methods or strategies for the user to work on these issues.
          3. Maintain an empathetic, understanding tone.
          4. If there is insufficient information to conclude on something, mention that gently.
          """

        formatted_history = self._format_history()
        report_prompt = ChatPromptTemplate.from_messages([
            ("system", SUMMARY_PROMPT.replace("{history}", formatted_history))
        ])

        summary_chain = (
           {
                "input": RunnablePassthrough()
            }
            | report_prompt
            | self.rag_llm
            | StrOutputParser()
        )

        report = await summary_chain.ainvoke("")
        return report


In [12]:
rag_sys = PsychoRag(youtube_data_path)

In [ ]:
answer = await rag_sys.ask("hello.")
print(answer)

Hello! Nice to meet you. How are you doing today?


In [ ]:
answer = await rag_sys.ask("What kind of meditation would you suggest for me?")

print(answer)

Thank you for asking about meditation! It's a great topic.

Based on our conversation, I understand that you're looking for a meditation technique that aligns with your goals and preferences. From what you've shared, it seems like you're interested in exploring meditation as a way to better understand your thoughts, emotions, and behaviors.

Before suggesting a specific meditation technique, I'd like to clarify a few things. You mentioned that you've been thinking really fast and have a tendency to get lost in your thoughts. You also mentioned that you're interested in controlling your emotions and becoming more aware of your true self.

From my understanding, you might benefit from a meditation technique that helps you cultivate mindfulness, clarity, and emotional regulation. Here are a few options you might find helpful:

1. **Body Scan Meditation**: This technique involves bringing your attention to different parts of your body, starting from your toes and moving up to the top of yo

In [ ]:
report = await rag_sys.end_session()
print(report)

Based on our conversation, it seems that the user's main psychological concerns and issues that came up are:

1. Difficulty controlling their thoughts and emotions: The user mentioned that they tend to think really fast and get lost in their thoughts, which can be overwhelming. They're also interested in learning to regulate their emotions and develop a greater sense of self-awareness.

It appears that the user is struggling to manage their mental chatter and emotions, which can impact their daily life and overall well-being.

Possible methods or strategies for the user to work on these issues could be:

1. Mindfulness-based practices: The user may benefit from mindfulness-based meditation techniques, such as body scan meditation, loving-kindness meditation, or mindful breathing. These practices can help them cultivate greater awareness of their thoughts, emotions, and physical sensations, allowing them to better manage their mental chatter and emotions.

I'd like to emphasize that eve

# RAG Evaluation

In [13]:
qa_ds_path = '/content/drive/MyDrive/itmo-things/psycho_questions_dataset'

In [14]:
questions = []
ground_truths = []

for root, dirs, files in os.walk(qa_ds_path):
  for file in sorted(files):

    if file.split('_')[0] == 'question':
      with open(os.path.join(root, file), 'r') as f:
        file_content = f.read()
        questions.append(file_content)

    elif file.split('_')[0] == 'answer':
      with open(os.path.join(root, file), 'r') as f:
        file_content = f.read()
        ground_truths.append(file_content)


In [15]:
print(len(questions), len(ground_truths))


25 25


In [17]:

answers = []
contexts = []
err = []

for i, query in enumerate(questions):
  try:
    answer = await rag_sys.ask(query)
    answers.append(answer)
    contexts.append([docs.page_content for docs in rag_sys.retriever.get_relevant_documents(query)])
    print(f'Question asked: {query[:50]}...')
  except:
    err.append(i)
    print(f'Error for question {i+1} occured: {query[:50]}...')

<ipython-input-17-cc51bab7e62f>:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([docs.page_content for docs in rag_sys.retriever.get_relevant_documents(query)])


Question asked: Why do bad people rarely realize they're bad, and ...
Question asked: What signs or symptoms might you see from a child ...
Question asked: Is there really no treatment/therapy to help narci...
Question asked: Why do autistic people so often have adhd?

I've n...
Error for question 5 occured: Is it possible to display stereotypically autistic...
Question asked: Is it possible for trauma to calcify you until you...
Error for question 7 occured: Is it possible for your brain to forget about thin...
Error for question 8 occured: What kind of things cause someone to become self-c...
Error for question 9 occured: Therapists: can you tell if your patient is toxic/...
Error for question 10 occured: What is self soothing and how do people regulate t...
Error for question 11 occured: What is self soothing and how do people regulate t...
Error for question 12 occured: Why some people don’t tell when they are mad at yo...
Error for question 13 occured: What's the difference betwee

In [20]:
questions_c = [questions[i] for i in range(len(questions)) if i not in err]
ground_truths_c = [ground_truths[i] for i in range(len(ground_truths)) if i not in err]

data = {
    "question": questions_c,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths_c
}

data['answer'][0]

'What a thought-provoking question! As a psychological assistant, I\'ll try to provide some insights based on the context you\'ve shared.\n\nIt\'s not uncommon for people who exhibit self-centered, aggressive, and manipulative behaviors to be oblivious to their own flaws. This phenomenon can be attributed to various psychological factors, including:\n\n1. **Conditioning**: People often learn their behaviors and attitudes through conditioning, either from their environment, upbringing, or social interactions. This can lead to a sense of entitlement, narcissism, or aggression, which may not be recognized as problematic by the individual.\n2. **Lack of empathy**: Individuals who struggle with empathy might not be able to put themselves in others\' shoes, making it difficult for them to recognize the harm they cause. This lack of empathy can be a result of various factors, such as childhood experiences, personality traits, or neurological conditions.\n3. **Deflection and denial**: When con

In [34]:
from groqeval.evaluate import GroqEval

evaluator = GroqEval(api_key=os.environ['GROQ_API_KEY'])

score = .0
N = len(data['answer'])
for i in range(N):
  answer_relevance = evaluator("answer_relevance", prompt = data['question'][i], output = data['answer'][i])
  score += answer_relevance.score()['score']

print(f'Average relevancy score: {(score / N):.5f}')

[{'role': 'system', 'content': 'Please process the following output from a language model and decompose it into individual phrases or chunks. For each phrase or chunk, evaluate whether it can be considered a statement based on its form as a declarative construct that communicates information, opinions, or beliefs. A phrase should be marked as a statement (true) if it forms a clear, standalone declaration. Phrases that are overly vague, questions, or merely connective phrases without any declarative content should be marked as not statements (false). Return the results in a JSON format. The JSON should have an array of objects, each representing a phrase with two properties: a \'string\' that contains the phrase text, and a \'flag\' that is a boolean indicating whether the text is considered a statement (true) or not (false).\nUse the following JSON schema for your output: {\n  "$defs": {\n    "Sentence": {\n      "properties": {\n        "string": {\n          "title": "String",\n     

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [35]:
er = await rag_sys.ask('nice')

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jd99r3aefvb875yp0bgatc6h` on : Limit 500000, Used 492902, Requested 8586. Please try again in 4m17.0106s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [ ]:
# лимиты >:(